## 解压数据集

In [1]:
# !cd /home/aistudio/data/data29604 && unzip -q imgs.zip

In [2]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
# !ls /home/aistudio/data

## 读取标签，获得类别，创建空目录以便下一步移动数据

In [3]:
# 将'p081','p075'两位司机的数据作为验证集，并将其移动到data/valid/文件夹下
import os
import pandas as pd
import numpy as np
import shutil

root_dir = "/home/aistudio/"
data_dir = "/home/aistudio/data/data29604/"

driver_imgs_list_csv = os.path.join(root_dir, "work/driver_imgs_list.csv")

if not os.path.exists(root_dir + "data/eval_set"):
    os.mkdir(root_dir + "data/eval_set")
    # for i in range(10):
        # os.mkdir(root_dir + "data/eval_set/c%d"%i)

if not os.path.exists(root_dir + "data/train_set"):
    os.mkdir(root_dir + "data/train_set")
    # for i in range(10):
    #     os.mkdir(root_dir + "data/train_set/c%d"%i)

datafile = pd.read_csv(driver_imgs_list_csv)
drivers = datafile.subject.values
all_class_list = datafile.classname.values
# 去重
class_list = []
for id in all_class_list:
    if id not in class_list:
        class_list.append(id)
class_list = class_list[0:10]
print(class_list)
print(drivers)

valid_subjects = ['p081','p075']
train_subjects = list(set(drivers).difference(set(valid_subjects)))

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
['p002' 'p002' 'p002' ... 'p081' 'p081' 'p081']


## 将训练数据和测试数据复制到train_set and eval_set 目录内，并获取相应的标签txt文件

In [4]:
import codecs # codecs专门用作编码转换
import os
import random
import shutil
from PIL import Image
all_file_dir = "/home/aistudio/data/"
eval_image_dir = "/home/aistudio/data/eval_set/"
train_image_dir = "/home/aistudio/data/train_set/"
train_file = codecs.open(os.path.join(all_file_dir, "train_list.txt"), 'w')
eval_file = codecs.open(os.path.join(all_file_dir, "validate_list.txt"), 'w')
label_file = codecs.open(os.path.join(all_file_dir, "label_list.txt"), 'w')
for label in class_list:
    label_file.write("{0} {1}\n".format(label[1], label))

for valid_subject in valid_subjects:
    df_valid = datafile[(datafile["subject"]==valid_subject)]
    for index, row in df_valid.iterrows():
        subpath_s = row["classname"] + "/" + row["img"]
        subpath_g = row["img"]
        if os.path.exists(os.path.join(data_dir,"train",subpath_s)):
            shutil.copy(os.path.join(data_dir, "train", subpath_s),os.path.join(root_dir, "data/eval_set", subpath_g))
            final_class = int(row["classname"][1])
            # if final_class > 0 and final_class<5:
            #     final_class = 1
            # elif final_class > 4 and final_class != 9:
            #     final_class = final_class-3
            # else:
            #     final_class = 0
            eval_file.write("{0} {1}\n".format(os.path.join(eval_image_dir, subpath_g), final_class))
        else:
            print("cannot copy {} : {}".format(row["subject"],subpath_s))

for train_subject in train_subjects:
    df_train = datafile[(datafile["subject"] == train_subject)]
    for index, row in df_train.iterrows():
        subpath_s = row["classname"] + "/" + row["img"]
        subpath_g = row["img"]
        if os.path.exists(os.path.join(data_dir,"train",subpath_s)):
            shutil.copy(os.path.join(data_dir, "train", subpath_s),os.path.join(root_dir, "data/train_set", subpath_g))
            final_class = int(row["classname"][1])
            # if final_class > 0 and final_class<5:
            #     final_class = 1
            # elif final_class > 4 and final_class != 9:
            #     final_class = final_class-3
            # else:
            #     final_class = 0
            train_file.write("{0} {1}\n".format(os.path.join(train_image_dir, subpath_g), final_class))
        else:
            print("cannot copy {} : {}".format(row["subject"],subpath_s))

train_file.close()
eval_file.close()

In [ ]:
import paddlehub as hub

#创建日志文件，储存lenet训练结果
# log_writer = LogWriter("./log/vgg16")

module= hub.Module(name="vgg16_imagenet")
# module = hub.Module(name="vgg19_imagenet")
# module = hub.Module(name="resnet_v2_50_imagenet")
# module =hub.Module(name="mobilenet_v2_imagenet")

input_dict, output_dict, program = module.context(trainable=True)

[2020-06-15 18:54:14,661] [    INFO] - Installing vgg16_imagenet module
[2020-06-15 18:54:14,666] [    INFO] - Module vgg16_imagenet already installed in /home/aistudio/.paddlehub/modules/vgg16_imagenet
[2020-06-15 18:54:15,087] [    INFO] - 32 pretrained paramaters loaded by PaddleHub


In [ ]:
from paddlehub.dataset.base_cv_dataset import BaseCVDataset
   
class DriverDataset(BaseCVDataset):	
   def __init__(self):	
       # 数据集存放位置
       self.dataset_dir = "/home/aistudio/data"
       super(DriverDataset, self).__init__(
           base_path=self.dataset_dir,
           train_list_file="train_list.txt",
           validate_list_file="validate_list.txt",
        #    test_list_file="test_list.txt",
        #    predict_file="predict_list.txt",
        #    label_list_file="label_list.txt",
           label_list=["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]
           )
dataset = DriverDataset()

In [ ]:
strategy = hub.DefaultFinetuneStrategy(
    learning_rate=1e-2,
    optimizer_name="adam",
    regularization_coeff=0.1)

config = hub.RunConfig(use_cuda=True, use_data_parallel=True, num_epoch=20, batch_size=32, strategy=strategy)

[2020-06-15 18:54:15,138] [    INFO] - Checkpoint dir: ckpt_20200615185415


In [ ]:
data_reader = hub.reader.ImageClassificationReader(
    image_width=module.get_expected_image_width(),
    image_height=module.get_expected_image_height(),
    images_mean=module.get_pretrained_images_mean(),
    images_std=module.get_pretrained_images_std(),
    dataset=dataset)

[2020-06-15 18:54:15,144] [    INFO] - Dataset label map = {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


In [ ]:
import logging
import os

def init_log_config():
    """
    初始化日志相关配置
    :return:
    """
    global logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    log_path = os.path.join(os.getcwd(), 'logs')
    if not os.path.exists(log_path):
        os.makedirs(log_path)
    log_name = os.path.join(log_path, 'train.log')
    sh = logging.StreamHandler()
    fh = logging.FileHandler(log_name, mode='w')
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter("%(message)s")
    fh.setFormatter(formatter)
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    logger.addHandler(fh)

init_log_config()

In [ ]:
from paddlehub.common.logger import logger
from paddlehub.finetune.checkpoint import load_checkpoint, save_checkpoint

feature_map = output_dict["feature_map"]
feed_list = [input_dict["image"].name]
from tb_paddle import SummaryWriter
tb_writer = SummaryWriter("./logs/vgg19")
class MyClassifierTask(hub.ImageClassifierTask):
    def _build_env_start_event(self):
        self.stepmy = 0
    def _log_interval_event(self, run_states):
        self.stepmy = self.stepmy+1
        score, avg_loss, run_speed = self._calculate_metrics(run_states)
        avg_acc = score['acc']
        logger.info("{}\t{}\t{}\t{}".format(self.stepmy,avg_acc,avg_loss,run_speed))
        tb_writer.add_scalar(
            tag="Loss_{}".format(self.phase),
            scalar_value=avg_loss,
            global_step=self._envs['train'].current_step)
        tb_writer.add_scalar(
            tag="Acc_{}".format(self.phase),
            scalar_value=avg_acc,
            global_step=self._envs['train'].current_step)


task = MyClassifierTask(
    data_reader=data_reader,
    feed_list=feed_list,
    feature=feature_map,
    num_classes=dataset.num_labels,
    config=config
    )

task.finetune_and_eval()

[2020-06-15 18:54:18,072] [    INFO] - Strategy with slanted triangle learning rate, L2 regularization, 
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:804: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)
[2020-06-15 18:54:20,290] [    INFO] - Try loading checkpoint from ckpt_20200615185415/ckpt.meta
[2020-06-15 18:54:20,291] [    INFO] - PaddleHub model checkpoint not found, start from scratch...
[2020-06-15 18:54:20,302] [    INFO] - PaddleHub finetune start


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# !cp /home/aistudio/work/ResNet50_pretrained.tar /home/aistudio/data/ResNet50_pretrained.tar
# !cd data && tar -xf ResNet50_pretrained.tar

In [ ]:
# 解压数据集
# import os
# import zipfile
# os.chdir('/home/aistudio/data/data29604')
# extracting = zipfile.ZipFile('imgs.zip')
# extracting.extractall()

# VisualDL

In [ ]:
# import paddle
# paddle.__version__

In [ ]:
# import numpy as np
# from PIL import Image
# from visualdl import LogWriter
# import os

# #确保路径为'/home/aistudio'
# os.chdir('/home/aistudio')

# #创建 LogWriter 对象，将图像数据存放在 `./log/train`路径下
# from visualdl import LogWriter
# log_writer = LogWriter("./log/train",sync_cycle=10)


In [ ]:
# # -*- coding: UTF-8 -*-
# """
# 训练常用视觉基础网络，用于分类任务
# 需要将训练图片，类别文件 label_list.txt 放置在同一个文件夹下
# 程序会先读取 train.txt 文件获取类别数和图片数量
# """
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# import os
# import numpy as np
# import time
# import math
# import paddle
# import paddle.fluid as fluid
# import codecs
# import logging

# from paddle.fluid.initializer import MSRA
# from paddle.fluid.initializer import Uniform
# from paddle.fluid.param_attr import ParamAttr
# from PIL import Image
# from PIL import ImageEnhance
# import paddlehub as hub
# #使用VisualDL观察Paddle和手写两种模型进行训练对比
# import paddle.fluid.dygraph as dygraph
# from visualdl import LogWriter


# train_parameters = {
#     "input_size": [3, 224, 224],
#     "class_dim": -1,  # 分类数，会在初始化自定义 reader 的时候获得
#     "image_count": -1,  # 训练图片数量，会在初始化自定义 reader 的时候获得
#     "label_dict": {},
#     "data_dir": "data",  # 训练数据存储地址
#     "train_file_list": "train.txt",
#     "label_file": "label_list.txt",
#     "save_freeze_dir": "./freeze_model/resnet50",
#     "save_persistable_dir": "./persistable_params/50",
#     "continue_train": True,        # 是否接着上一次保存的参数接着训练，优先级高于预训练模型
#     "pretrained": False,            # 是否使用预训练的模型，对于inceptionv4模型暂无预训练参数
#     "pretrained_dir":"./data/ResNet50_pretrained", 
#     "mode": "train",
#     "num_epochs": 100,
#     "train_batch_size": 96,
#     "mean_rgb": [127.5, 127.5, 127.5],  # 常用图片的三通道均值，通常来说需要先对训练数据做统计，此处仅取中间值
#     "use_gpu": True,
#     "image_enhance_strategy": {  # 图像增强相关策略
#         "need_distort": True,  # 是否启用图像颜色增强
#         "need_rotate": True,   # 是否需要增加随机角度
#         "need_crop": True,      # 是否要增加裁剪
#         "need_flip": True,      # 是否要增加水平随机翻转
#         "hue_prob": 0.5,
#         "hue_delta": 18,
#         "contrast_prob": 0.5,
#         "contrast_delta": 0.5,
#         "saturation_prob": 0.5,
#         "saturation_delta": 0.5,
#         "brightness_prob": 0.5,
#         "brightness_delta": 0.125
#     },
#     "early_stop": {
#         "sample_frequency": 500,
#         "successive_limit": 5,
#         "good_acc1": 0.92
#     },
#     "rsm_strategy": {
#         "learning_rate": 0.001,
#         "lr_epochs": [20, 40, 60, 80, 100],
#         "lr_decay": [1, 0.5, 0.25, 0.1, 0.01, 0.002]
#     },
#     "momentum_strategy": {
#         "learning_rate": 0.002,
#         "lr_epochs": [20, 40, 60, 80, 100],
#         "lr_decay": [1, 0.5, 0.25, 0.1, 0.01, 0.002]
#     },
#     "sgd_strategy": {
#         "learning_rate": 0.002,
#         "lr_epochs": [20, 40, 60, 80, 100],
#         "lr_decay": [1, 0.5, 0.25, 0.1, 0.01, 0.002]
#     },
#     "adam_strategy": {
#         "learning_rate": 0.002
#     }
# }

# # 定义残差神经网络（ResNet）
# class ResNet50():
#     def __init__(self):
#         pass
    
#     def name(self):
#         """
#         返回网络名字
#         :return:
#         """
#         return 'ResNet50'

#     def net(self, input, class_dim=1000):
#         def conv_bn_layer(input, num_filters, filter_size, stride=1, groups=1, act=None, name=None):
#             conv = fluid.layers.conv2d(input=input,
#                                     num_filters=num_filters,
#                                     filter_size=filter_size,
#                                     stride=stride,
#                                     padding=(filter_size - 1) // 2,
#                                     groups=groups,
#                                     act=None,
#                                     param_attr=ParamAttr(name=name + "_weights"),
#                                     bias_attr=False,
#                                     name=name + '.conv2d.output.1')
#             if name == "conv1":
#                 bn_name = "bn_" + name
#             else:
#                 bn_name = "bn" + name[3:]
#             return fluid.layers.batch_norm(input=conv,
#                                         act=act,
#                                         name=bn_name + '.output.1',
#                                         param_attr=ParamAttr(name=bn_name + '_scale'),
#                                         bias_attr=ParamAttr(bn_name + '_offset'),
#                                         moving_mean_name=bn_name + '_mean',
#                                         moving_variance_name=bn_name + '_variance', )

#         def shortcut(input, ch_out, stride, name):
#             ch_in = input.shape[1]
#             if ch_in != ch_out or stride != 1:
#                 return conv_bn_layer(input, ch_out, 1, stride, name=name)
#             else:
#                 return input

#         def bottleneck_block(input, num_filters, stride, name):
#             conv0 = conv_bn_layer(input=input,
#                                 num_filters=num_filters,
#                                 filter_size=1,
#                                 act='relu',
#                                 name=name + "_branch2a")
#             conv1 = conv_bn_layer(input=conv0,
#                                 num_filters=num_filters,
#                                 filter_size=3,
#                                 stride=stride,
#                                 act='relu',
#                                 name=name + "_branch2b")
#             conv2 = conv_bn_layer(input=conv1,
#                                 num_filters=num_filters * 4,
#                                 filter_size=1,
#                                 act=None,
#                                 name=name + "_branch2c")

#             short = shortcut(input, num_filters * 4, stride, name=name + "_branch1")

#             return fluid.layers.elementwise_add(x=short, y=conv2, act='relu', name=name + ".add.output.5")

#         depth = [3, 4, 6, 3]
#         num_filters = [64, 128, 256, 512]

#         conv = conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2, act='relu', name="conv1")
#         conv = fluid.layers.pool2d(input=conv, pool_size=3, pool_stride=2, pool_padding=1, pool_type='max')

#         for block in range(len(depth)):
#             for i in range(depth[block]):
#                 conv_name = "res" + str(block + 2) + chr(97 + i)
#                 conv = bottleneck_block(input=conv,
#                                         num_filters=num_filters[block],
#                                         stride=2 if i == 0 and block != 0 else 1,
#                                         name=conv_name)

#         pool = fluid.layers.pool2d(input=conv, pool_size=7, pool_type='avg', global_pooling=True)
#         # 停止梯度下降
#         pool.stop_gradient = True
#         # 这里再重新加载网络的分类器，大小为本项目的分类大小
#         out = fluid.layers.fc(input=pool, size=class_dim, act='softmax')
#         return out


# class InceptionV4():
#     def __init__(self):
#         pass
    
#     def name(self):
#         """
#         返回网络名字
#         :return:
#         """
#         return 'InceptionV4'

#     def net(self, input, class_dim=1000):
#         x = self.inception_stem(input)

#         for i in range(4):
#             x = self.inceptionA(x, name=str(i + 1))
#         x = self.reductionA(x)

#         for i in range(7):
#             x = self.inceptionB(x, name=str(i + 1))
#         x = self.reductionB(x)

#         for i in range(3):
#             x = self.inceptionC(x, name=str(i + 1))

#         pool = fluid.layers.pool2d(
#             input=x, pool_size=8, pool_type='avg', global_pooling=True)

#         drop = fluid.layers.dropout(x=pool, dropout_prob=0.2)

#         stdv = 1.0 / math.sqrt(drop.shape[1] * 1.0)
#         out = fluid.layers.fc(
#             input=drop,
#             size=class_dim,
#             act='softmax',
#             param_attr=ParamAttr(
#                 initializer=fluid.initializer.Uniform(-stdv, stdv),
#                 name="final_fc_weights"),
#             bias_attr=ParamAttr(
#                 initializer=fluid.initializer.Uniform(-stdv, stdv),
#                 name="final_fc_offset"))
#         return out

#     def conv_bn_layer(self,
#                       data,
#                       num_filters,
#                       filter_size,
#                       stride=1,
#                       padding=0,
#                       groups=1,
#                       act='relu',
#                       name=None):
#         conv = fluid.layers.conv2d(
#             input=data,
#             num_filters=num_filters,
#             filter_size=filter_size,
#             stride=stride,
#             padding=padding,
#             groups=groups,
#             act=None,
#             param_attr=ParamAttr(name=name + "_weights"),
#             bias_attr=False,
#             name=name)
#         bn_name = name + "_bn"
#         return fluid.layers.batch_norm(
#             input=conv,
#             act=act,
#             name=bn_name,
#             param_attr=ParamAttr(name=bn_name + "_scale"),
#             bias_attr=ParamAttr(name=bn_name + "_offset"),
#             moving_mean_name=bn_name + '_mean',
#             moving_variance_name=bn_name + '_variance')

#     def inception_stem(self, data, name=None):
#         conv = self.conv_bn_layer(
#             data, 32, 3, stride=2, act='relu', name="conv1_3x3_s2")
#         conv = self.conv_bn_layer(conv, 32, 3, act='relu', name="conv2_3x3_s1")
#         conv = self.conv_bn_layer(
#             conv, 64, 3, padding=1, act='relu', name="conv3_3x3_s1")

#         pool1 = fluid.layers.pool2d(
#             input=conv, pool_size=3, pool_stride=2, pool_type='max')
#         conv2 = self.conv_bn_layer(
#             conv, 96, 3, stride=2, act='relu', name="inception_stem1_3x3_s2")
#         concat = fluid.layers.concat([pool1, conv2], axis=1)

#         conv1 = self.conv_bn_layer(
#             concat, 64, 1, act='relu', name="inception_stem2_3x3_reduce")
#         conv1 = self.conv_bn_layer(
#             conv1, 96, 3, act='relu', name="inception_stem2_3x3")

#         conv2 = self.conv_bn_layer(
#             concat, 64, 1, act='relu', name="inception_stem2_1x7_reduce")
#         conv2 = self.conv_bn_layer(
#             conv2,
#             64, (7, 1),
#             padding=(3, 0),
#             act='relu',
#             name="inception_stem2_1x7")
#         conv2 = self.conv_bn_layer(
#             conv2,
#             64, (1, 7),
#             padding=(0, 3),
#             act='relu',
#             name="inception_stem2_7x1")
#         conv2 = self.conv_bn_layer(
#             conv2, 96, 3, act='relu', name="inception_stem2_3x3_2")

#         concat = fluid.layers.concat([conv1, conv2], axis=1)

#         conv1 = self.conv_bn_layer(
#             concat, 192, 3, stride=2, act='relu', name="inception_stem3_3x3_s2")
#         pool1 = fluid.layers.pool2d(
#             input=concat, pool_size=3, pool_stride=2, pool_type='max')

#         concat = fluid.layers.concat([conv1, pool1], axis=1)

#         return concat

#     def inceptionA(self, data, name=None):
#         pool1 = fluid.layers.pool2d(
#             input=data, pool_size=3, pool_padding=1, pool_type='avg')
#         conv1 = self.conv_bn_layer(
#             pool1, 96, 1, act='relu', name="inception_a" + name + "_1x1")

#         conv2 = self.conv_bn_layer(
#             data, 96, 1, act='relu', name="inception_a" + name + "_1x1_2")

#         conv3 = self.conv_bn_layer(
#             data, 64, 1, act='relu', name="inception_a" + name + "_3x3_reduce")
#         conv3 = self.conv_bn_layer(
#             conv3,
#             96,
#             3,
#             padding=1,
#             act='relu',
#             name="inception_a" + name + "_3x3")

#         conv4 = self.conv_bn_layer(
#             data,
#             64,
#             1,
#             act='relu',
#             name="inception_a" + name + "_3x3_2_reduce")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             96,
#             3,
#             padding=1,
#             act='relu',
#             name="inception_a" + name + "_3x3_2")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             96,
#             3,
#             padding=1,
#             act='relu',
#             name="inception_a" + name + "_3x3_3")

#         concat = fluid.layers.concat([conv1, conv2, conv3, conv4], axis=1)

#         return concat

#     def reductionA(self, data, name=None):
#         pool1 = fluid.layers.pool2d(
#             input=data, pool_size=3, pool_stride=2, pool_type='max')

#         conv2 = self.conv_bn_layer(
#             data, 384, 3, stride=2, act='relu', name="reduction_a_3x3")

#         conv3 = self.conv_bn_layer(
#             data, 192, 1, act='relu', name="reduction_a_3x3_2_reduce")
#         conv3 = self.conv_bn_layer(
#             conv3, 224, 3, padding=1, act='relu', name="reduction_a_3x3_2")
#         conv3 = self.conv_bn_layer(
#             conv3, 256, 3, stride=2, act='relu', name="reduction_a_3x3_3")

#         concat = fluid.layers.concat([pool1, conv2, conv3], axis=1)

#         return concat

#     def inceptionB(self, data, name=None):
#         pool1 = fluid.layers.pool2d(
#             input=data, pool_size=3, pool_padding=1, pool_type='avg')
#         conv1 = self.conv_bn_layer(
#             pool1, 128, 1, act='relu', name="inception_b" + name + "_1x1")

#         conv2 = self.conv_bn_layer(
#             data, 384, 1, act='relu', name="inception_b" + name + "_1x1_2")

#         conv3 = self.conv_bn_layer(
#             data, 192, 1, act='relu', name="inception_b" + name + "_1x7_reduce")
#         conv3 = self.conv_bn_layer(
#             conv3,
#             224, (1, 7),
#             padding=(0, 3),
#             act='relu',
#             name="inception_b" + name + "_1x7")
#         conv3 = self.conv_bn_layer(
#             conv3,
#             256, (7, 1),
#             padding=(3, 0),
#             act='relu',
#             name="inception_b" + name + "_7x1")

#         conv4 = self.conv_bn_layer(
#             data,
#             192,
#             1,
#             act='relu',
#             name="inception_b" + name + "_7x1_2_reduce")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             192, (1, 7),
#             padding=(0, 3),
#             act='relu',
#             name="inception_b" + name + "_1x7_2")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             224, (7, 1),
#             padding=(3, 0),
#             act='relu',
#             name="inception_b" + name + "_7x1_2")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             224, (1, 7),
#             padding=(0, 3),
#             act='relu',
#             name="inception_b" + name + "_1x7_3")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             256, (7, 1),
#             padding=(3, 0),
#             act='relu',
#             name="inception_b" + name + "_7x1_3")

#         concat = fluid.layers.concat([conv1, conv2, conv3, conv4], axis=1)

#         return concat

#     def reductionB(self, data, name=None):
#         pool1 = fluid.layers.pool2d(
#             input=data, pool_size=3, pool_stride=2, pool_type='max')

#         conv2 = self.conv_bn_layer(
#             data, 192, 1, act='relu', name="reduction_b_3x3_reduce")
#         conv2 = self.conv_bn_layer(
#             conv2, 192, 3, stride=2, act='relu', name="reduction_b_3x3")

#         conv3 = self.conv_bn_layer(
#             data, 256, 1, act='relu', name="reduction_b_1x7_reduce")
#         conv3 = self.conv_bn_layer(
#             conv3,
#             256, (1, 7),
#             padding=(0, 3),
#             act='relu',
#             name="reduction_b_1x7")
#         conv3 = self.conv_bn_layer(
#             conv3,
#             320, (7, 1),
#             padding=(3, 0),
#             act='relu',
#             name="reduction_b_7x1")
#         conv3 = self.conv_bn_layer(
#             conv3, 320, 3, stride=2, act='relu', name="reduction_b_3x3_2")

#         concat = fluid.layers.concat([pool1, conv2, conv3], axis=1)

#         return concat

#     def inceptionC(self, data, name=None):
#         pool1 = fluid.layers.pool2d(
#             input=data, pool_size=3, pool_padding=1, pool_type='avg')
#         conv1 = self.conv_bn_layer(
#             pool1, 256, 1, act='relu', name="inception_c" + name + "_1x1")

#         conv2 = self.conv_bn_layer(
#             data, 256, 1, act='relu', name="inception_c" + name + "_1x1_2")

#         conv3 = self.conv_bn_layer(
#             data, 384, 1, act='relu', name="inception_c" + name + "_1x1_3")
#         conv3_1 = self.conv_bn_layer(
#             conv3,
#             256, (1, 3),
#             padding=(0, 1),
#             act='relu',
#             name="inception_c" + name + "_1x3")
#         conv3_2 = self.conv_bn_layer(
#             conv3,
#             256, (3, 1),
#             padding=(1, 0),
#             act='relu',
#             name="inception_c" + name + "_3x1")

#         conv4 = self.conv_bn_layer(
#             data, 384, 1, act='relu', name="inception_c" + name + "_1x1_4")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             448, (1, 3),
#             padding=(0, 1),
#             act='relu',
#             name="inception_c" + name + "_1x3_2")
#         conv4 = self.conv_bn_layer(
#             conv4,
#             512, (3, 1),
#             padding=(1, 0),
#             act='relu',
#             name="inception_c" + name + "_3x1_2")
#         conv4_1 = self.conv_bn_layer(
#             conv4,
#             256, (1, 3),
#             padding=(0, 1),
#             act='relu',
#             name="inception_c" + name + "_1x3_3")
#         conv4_2 = self.conv_bn_layer(
#             conv4,
#             256, (3, 1),
#             padding=(1, 0),
#             act='relu',
#             name="inception_c" + name + "_3x1_3")

#         concat = fluid.layers.concat(
#             [conv1, conv2, conv3_1, conv3_2, conv4_1, conv4_2], axis=1)

#         return concat


# def init_log_config():
#     """
#     初始化日志相关配置
#     :return:
#     """
#     global logger
#     logger = logging.getLogger()
#     logger.setLevel(logging.INFO)
#     log_path = os.path.join(os.getcwd(), 'logs')
#     if not os.path.exists(log_path):
#         os.makedirs(log_path)
#     log_name = os.path.join(log_path, 'train.log')
#     sh = logging.StreamHandler()
#     fh = logging.FileHandler(log_name, mode='w')
#     fh.setLevel(logging.DEBUG)
#     formatter = logging.Formatter("%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s")
#     fh.setFormatter(formatter)
#     sh.setFormatter(formatter)
#     logger.addHandler(sh)
#     logger.addHandler(fh)


# def init_train_parameters():
#     """
#     初始化训练参数，主要是初始化图片数量，类别数
#     :return:
#     """
#     train_file_list = os.path.join(train_parameters['data_dir'], train_parameters['train_file_list'])
#     label_list = os.path.join(train_parameters['data_dir'], train_parameters['label_file'])
#     index = 0
#     with codecs.open(label_list, encoding='utf-8') as flist:
#         lines = [line.strip() for line in flist]
#         for line in lines:
#             parts = line.strip().split()
#             train_parameters['label_dict'][parts[1]] = int(parts[0])
#             index += 1
#         train_parameters['class_dim'] = index
#     with codecs.open(train_file_list, encoding='utf-8') as flist:
#         lines = [line.strip() for line in flist]
#         train_parameters['image_count'] = len(lines)


# def resize_img(img, target_size):
#     """
#     强制缩放图片
#     :param img:
#     :param target_size:
#     :return:
#     """
#     target_size = input_size
#     img = img.resize((target_size[1], target_size[2]), Image.BILINEAR)
#     return img


# def random_crop(img, scale=[0.08, 1.0], ratio=[3. / 4., 4. / 3.]):
#     aspect_ratio = math.sqrt(np.random.uniform(*ratio))
#     w = 1. * aspect_ratio
#     h = 1. / aspect_ratio

#     bound = min((float(img.size[0]) / img.size[1]) / (w**2),
#                 (float(img.size[1]) / img.size[0]) / (h**2))
#     scale_max = min(scale[1], bound)
#     scale_min = min(scale[0], bound)

#     target_area = img.size[0] * img.size[1] * np.random.uniform(scale_min,
#                                                                 scale_max)
#     target_size = math.sqrt(target_area)
#     w = int(target_size * w)
#     h = int(target_size * h)

#     i = np.random.randint(0, img.size[0] - w + 1)
#     j = np.random.randint(0, img.size[1] - h + 1)

#     img = img.crop((i, j, i + w, j + h))
#     img = img.resize((train_parameters['input_size'][1], train_parameters['input_size'][2]), Image.BILINEAR)
#     return img


# def rotate_image(img):
#     """
#     图像增强，增加随机旋转角度
#     """
#     angle = np.random.randint(-14, 15)
#     img = img.rotate(angle)
#     return img


# def random_brightness(img):
#     """
#     图像增强，亮度调整
#     :param img:
#     :return:
#     """
#     prob = np.random.uniform(0, 1)
#     if prob < train_parameters['image_enhance_strategy']['brightness_prob']:
#         brightness_delta = train_parameters['image_enhance_strategy']['brightness_delta']
#         delta = np.random.uniform(-brightness_delta, brightness_delta) + 1
#         img = ImageEnhance.Brightness(img).enhance(delta)
#     return img


# def random_contrast(img):
#     """
#     图像增强，对比度调整
#     :param img:
#     :return:
#     """
#     prob = np.random.uniform(0, 1)
#     if prob < train_parameters['image_enhance_strategy']['contrast_prob']:
#         contrast_delta = train_parameters['image_enhance_strategy']['contrast_delta']
#         delta = np.random.uniform(-contrast_delta, contrast_delta) + 1
#         img = ImageEnhance.Contrast(img).enhance(delta)
#     return img


# def random_saturation(img):
#     """
#     图像增强，饱和度调整
#     :param img:
#     :return:
#     """
#     prob = np.random.uniform(0, 1)
#     if prob < train_parameters['image_enhance_strategy']['saturation_prob']:
#         saturation_delta = train_parameters['image_enhance_strategy']['saturation_delta']
#         delta = np.random.uniform(-saturation_delta, saturation_delta) + 1
#         img = ImageEnhance.Color(img).enhance(delta)
#     return img


# def random_hue(img):
#     """
#     图像增强，色度调整
#     :param img:
#     :return:
#     """
#     prob = np.random.uniform(0, 1)
#     if prob < train_parameters['image_enhance_strategy']['hue_prob']:
#         hue_delta = train_parameters['image_enhance_strategy']['hue_delta']
#         delta = np.random.uniform(-hue_delta, hue_delta)
#         img_hsv = np.array(img.convert('HSV'))
#         img_hsv[:, :, 0] = img_hsv[:, :, 0] + delta
#         img = Image.fromarray(img_hsv, mode='HSV').convert('RGB')
#     return img


# def distort_color(img):
#     """
#     概率的图像增强
#     :param img:
#     :return:
#     """
#     prob = np.random.uniform(0, 1)
#     # Apply different distort order
#     if prob < 0.35:
#         img = random_brightness(img)
#         img = random_contrast(img)
#         img = random_saturation(img)
#         img = random_hue(img)
#     elif prob < 0.7:
#         img = random_brightness(img)
#         img = random_saturation(img)
#         img = random_hue(img)
#         img = random_contrast(img)
#     return img


# def custom_image_reader(file_list, data_dir, mode):
#     """
#     自定义用户图片读取器，先初始化图片种类，数量
#     :param file_list:
#     :param data_dir:
#     :param mode:
#     :return:
#     """
#     with codecs.open(file_list) as flist:
#         lines = [line.strip() for line in flist]

#     def reader():
#         np.random.shuffle(lines)
#         for line in lines:
#             if mode == 'train' or mode == 'val':
#                 img_path, label = line.split()
#                 img = Image.open(img_path)
#                 try:
#                     if img.mode != 'RGB':
#                         img = img.convert('RGB')
#                     if train_parameters['image_enhance_strategy']['need_distort'] == True:
#                         img = distort_color(img)
#                     if train_parameters['image_enhance_strategy']['need_rotate'] == True:
#                         img = rotate_image(img)
#                     if train_parameters['image_enhance_strategy']['need_crop'] == True:
#                         img = random_crop(img, train_parameters['input_size'])
#                     if train_parameters['image_enhance_strategy']['need_flip'] == True:
#                         mirror = int(np.random.uniform(0, 2))
#                         if mirror == 1:
#                             img = img.transpose(Image.FLIP_LEFT_RIGHT)
#                     # HWC--->CHW && normalized
#                     img = np.array(img).astype('float32')
#                     img -= train_parameters['mean_rgb']
#                     img = img.transpose((2, 0, 1))  # HWC to CHW
#                     img *= 0.007843                 # 像素值归一化
#                     yield img, int(label)
#                 except Exception as e:
#                     pass                            # 以防某些图片读取处理出错，加异常处理
#             elif mode == 'test':
#                 img_path = os.path.join(data_dir, line)
#                 img = Image.open(img_path)
#                 if img.mode != 'RGB':
#                     img = img.convert('RGB')
#                 img = resize_img(img, train_parameters['input_size'])
#                 # HWC--->CHW && normalized
#                 img = np.array(img).astype('float32')
#                 img -= train_parameters['mean_rgb']
#                 img = img.transpose((2, 0, 1))  # HWC to CHW
#                 img *= 0.007843  # 像素值归一化
#                 yield img

#     return reader


# def optimizer_momentum_setting():
#     """
#     阶梯型的学习率适合比较大规模的训练数据
#     """
#     learning_strategy = train_parameters['momentum_strategy']
#     batch_size = train_parameters["train_batch_size"]
#     iters = train_parameters["image_count"] // batch_size
#     lr = learning_strategy['learning_rate']

#     boundaries = [i * iters for i in learning_strategy["lr_epochs"]]
#     values = [i * lr for i in learning_strategy["lr_decay"]]
#     learning_rate = fluid.layers.piecewise_decay(boundaries, values)
#     optimizer = fluid.optimizer.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
#     return optimizer


# def optimizer_rms_setting():
#     """
#     阶梯型的学习率适合比较大规模的训练数据
#     """
#     batch_size = train_parameters["train_batch_size"]
#     iters = train_parameters["image_count"] // batch_size
#     learning_strategy = train_parameters['rsm_strategy']
#     lr = learning_strategy['learning_rate']

#     boundaries = [i * iters for i in learning_strategy["lr_epochs"]]
#     values = [i * lr for i in learning_strategy["lr_decay"]]

#     optimizer = fluid.optimizer.RMSProp(
#         learning_rate=fluid.layers.piecewise_decay(boundaries, values))

#     return optimizer


# def optimizer_sgd_setting():
#     """
#     loss下降相对较慢，但是最终效果不错，阶梯型的学习率适合比较大规模的训练数据
#     """
#     learning_strategy = train_parameters['sgd_strategy']
#     batch_size = train_parameters["train_batch_size"]
#     iters = train_parameters["image_count"] // batch_size
#     lr = learning_strategy['learning_rate']

#     boundaries = [i * iters for i in learning_strategy["lr_epochs"]]
#     values = [i * lr for i in learning_strategy["lr_decay"]]
#     learning_rate = fluid.layers.piecewise_decay(boundaries, values)
#     optimizer = fluid.optimizer.SGD(learning_rate=learning_rate)
#     return optimizer


# def optimizer_adam_setting():
#     """
#     能够比较快速的降低 loss，但是相对后期乏力
#     """
#     learning_strategy = train_parameters['adam_strategy']
#     learning_rate = learning_strategy['learning_rate']
#     optimizer = fluid.optimizer.Adam(learning_rate=learning_rate)
#     return optimizer


# def load_params(exe, program):
#     if train_parameters['continue_train'] and os.path.exists(train_parameters['save_persistable_dir']):
#         logger.info('load params from retrain model')
#         fluid.io.load_persistables(executor=exe,
#                                    dirname=train_parameters['save_persistable_dir'],
#                                    main_program=program)
#     elif train_parameters['pretrained'] and os.path.exists(train_parameters['pretrained_dir']):
#         logger.info('load params from pretrained model')
#         def if_exist(var):
#             return os.path.exists(os.path.join(train_parameters['pretrained_dir'], var.name))

#         fluid.io.load_vars(exe, train_parameters['pretrained_dir'], main_program=program,
#                            predicate=if_exist)


# def train():


#     EPOCH_NUM = train_parameters["num_epochs"]
#     #定义分别存储Paddle和python模型的训练信息的多维数组, 先存Paddle编写的模型的信息，后存python编写模型的信息
#     train_loss_acc = np.empty([2,EPOCH_NUM], dtype = float) #用二维数组分别存储两种模型的train_loss、test_loss
#     #定义VisualDL使用的变量。train和test集的loss定义为scalar类型（折线图），weight定义为histogram类型（直方图）
#     # log_writter = LogWriter("./log", sync_cycle=1000) #定义日志写入器。参数为日志存储位置和写入硬盘的数据量间隔
#     # with log_writter.mode("train") as logger: #变量定义在logger下操作。设置日志写入模式为“train”
#     #     train_loss_paddle = logger.scalar("Train Loss") #记录paddle模型train集loss
#     #     train_acc_paddle = logger.scalar("Train Acc") #记录paddle模型train集loss

#     train_prog = fluid.Program()
#     train_startup = fluid.Program()
#     logger.info("create prog success")
#     logger.info("train config: %s", str(train_parameters))
#     logger.info("build input custom reader and data feeder")
#     file_list = os.path.join(train_parameters['data_dir'], "train.txt")
#     mode = train_parameters['mode']
#     batch_reader = paddle.batch(custom_image_reader(file_list, train_parameters['data_dir'], mode),
#                                 batch_size=train_parameters['train_batch_size'],
#                                 drop_last=True)
#     place = fluid.CUDAPlace(0) if train_parameters['use_gpu'] else fluid.CPUPlace()
#     # 定义输入数据的占位符
#     img = fluid.layers.data(name='img', shape=train_parameters['input_size'], dtype='float32')
#     label = fluid.layers.data(name='label', shape=[1], dtype='int64')
#     feeder = fluid.DataFeeder(feed_list=[img, label], place=place)

#     # 选取不同的网络
#     logger.info("build newwork")
#     # model = InceptionV4()
#     model = ResNet50()
#     out = model.net(input=img, class_dim=train_parameters['class_dim'])
#     cost = fluid.layers.cross_entropy(out, label)
#     avg_cost = fluid.layers.mean(x=cost)
#     acc_top1 = fluid.layers.accuracy(input=out, label=label, k=1)
#     # 选取不同的优化器
#     # optimizer = optimizer_rms_setting()
#     # optimizer = optimizer_momentum_setting()
#     # optimizer = optimizer_sgd_setting()
#     optimizer = optimizer_adam_setting()
#     optimizer.minimize(avg_cost)
#     exe = fluid.Executor(place)

#     main_program = fluid.default_main_program()
#     exe.run(fluid.default_startup_program())
#     train_fetch_list = [avg_cost.name, acc_top1.name, out.name]
    
#     load_params(exe, main_program)

#     # 训练循环主体
#     stop_strategy = train_parameters['early_stop']
#     successive_limit = stop_strategy['successive_limit']
#     sample_freq = stop_strategy['sample_frequency']
#     good_acc1 = stop_strategy['good_acc1']
#     successive_count = 0
#     stop_train = False
#     total_batch_count = 0
#     for pass_id in range(train_parameters["num_epochs"]):
#         logger.info("current pass: %d, start read image", pass_id)
#         batch_id = 0

#         for step_id, data in enumerate(batch_reader()):
#             t1 = time.time()
#             # logger.info("data size:{0}".format(len(data)))
#             loss, acc1, pred_ot = exe.run(main_program,
#                                           feed=feeder.feed(data),
#                                           fetch_list=train_fetch_list)
#             t2 = time.time()
#             batch_id += 1
#             total_batch_count += 1
#             period = t2 - t1
#             loss = np.mean(np.array(loss))
#             acc1 = np.mean(np.array(acc1))
#             if batch_id % 100 == 0:
#                 logger.info("Pass {0}, trainbatch {1}, loss {2}, acc1 {3}, time {4}".format(pass_id, batch_id, loss, acc1,
#                                                                                             "%2.2f sec" % period))
#                 # print("Pass {0}, trainbatch {1}, loss {2}, acc1 {3}, time {4}".format(pass_id, batch_id, loss, acc1,
#                                                                                             # "%2.2f sec" % period))
#             # 简单的提前停止策略，认为连续达到某个准确率就可以停止了
#             if acc1 >= good_acc1:
#                 successive_count += 1
#                 logger.info("current acc1 {0} meets good {1}, successive count {2}".format(acc1, good_acc1, successive_count))
#                 fluid.io.save_inference_model(dirname=train_parameters['save_freeze_dir'],
#                                               feeded_var_names=['img'],
#                                               target_vars=[out],
#                                               main_program=main_program,
#                                               executor=exe)
#                 if successive_count >= successive_limit:
#                     logger.info("end training")
#                     stop_train = True
#                     break
#             else:
#                 successive_count = 0

#             # 通用的保存策略，减小意外停止的损失
#             if total_batch_count % sample_freq == 0:
#                 logger.info("temp save {0} batch train result, current acc1 {1}".format(total_batch_count, acc1))
#                 fluid.io.save_persistables(dirname=train_parameters['save_persistable_dir'],
#                                            main_program=main_program,
#                                            executor=exe)
#         train_loss_acc[0][pass_id] = loss
#         train_loss_acc[1][pass_id] = acc1
#         #  #写入Paddle模型的train和test的loss以及模型的权重weight到log文件夹下的对应文件中
#         # train_loss_paddle.add_record(pass_id, loss.numpy()[0])
#         # train_acc_paddle.add_record(pass_id, acc1.numpy()[0])
#         if stop_train:
#             break
#     logger.info("training till last epcho, end training")
#     fluid.io.save_persistables(dirname=train_parameters['save_persistable_dir'],
#                                            main_program=main_program,
#                                            executor=exe)
#     fluid.io.save_inference_model(dirname=train_parameters['save_freeze_dir'],
#                                               feeded_var_names=['img'],
#                                               target_vars=[out],
#                                               main_program=main_program.clone(for_test=True),
#                                               executor=exe)
# init_log_config()
# init_train_parameters()
# train()

# 模型评估

In [ ]:
# from __future__ import absolute_import   
# from __future__ import division   
# from __future__ import print_function   
   
# import os   
# import numpy as np   
# import random   
# import time   
# import codecs   
# import sys   
# import functools   
# import math   
# import paddle   
# import paddle.fluid as fluid   
# from paddle.fluid import core   
# from paddle.fluid.param_attr import ParamAttr   
# from PIL import Image, ImageEnhance   
   
# target_size = [3, 224, 224]   
# mean_rgb = [127.5, 127.5, 127.5]   
# data_dir = "data/"   
# eval_file = "eval.txt"   
# use_gpu = True   
# place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()   
# exe = fluid.Executor(place)   
# save_freeze_dir = "./freeze-model"   
# [inference_program, feed_target_names, fetch_targets] = fluid.io.load_inference_model(dirname=save_freeze_dir, executor=exe)   
# # print(fetch_targets)   
   
   
# def crop_image(img, target_size):   
#     width, height = img.size   
#     w_start = (width - target_size[2]) / 2   
#     h_start = (height - target_size[1]) / 2   
#     w_end = w_start + target_size[2]   
#     h_end = h_start + target_size[1]   
#     img = img.crop((w_start, h_start, w_end, h_end))   
#     return img   
   
   
# def resize_img(img, target_size):   
#     ret = img.resize((target_size[1], target_size[2]), Image.BILINEAR)   
#     return ret   
   
   
# def read_image(img_path):   
#     img = Image.open(img_path)   
#     if img.mode != 'RGB':   
#         img = img.convert('RGB')   
#     img = crop_image(img, target_size)   
#     img = np.array(img).astype('float32')   
#     img -= mean_rgb   
#     img = img.transpose((2, 0, 1))  # HWC to CHW   
#     img *= 0.007843   
#     img = img[np.newaxis,:]   
#     return img   
   
   
# def infer(image_path):   
#     tensor_img = read_image(image_path)   
#     label = exe.run(inference_program, feed={feed_target_names[0]: tensor_img}, fetch_list=fetch_targets)   
#     return np.argmax(label)   
   
   
# def eval_all():   
#     eval_file_path = os.path.join(data_dir, eval_file)   
#     total_count = 0   
#     right_count = 0   
#     with codecs.open(eval_file_path, encoding='utf-8') as flist:    
#         lines = [line.strip() for line in flist]   
#         t1 = time.time()   
#         for line in lines:   
#             total_count += 1   
#             parts = line.strip().split()   
#             result = infer(parts[0])   
#             # print("infer result:{0} answer:{1}".format(result, parts[1]))   
#             if str(result) == parts[1]:   
#                 right_count += 1   
#         period = time.time() - t1   
#         print("total eval count:{0} cost time:{1} predict accuracy:{2}".format(total_count, "%2.2f sec" % period, right_count / total_count))   
   
   
# if 1:   
#     eval_all() 

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 